# Embeddings

En un anterior post sobre [tokens](https://maximofn.com/tokens/), ya vimos la representación mínima de cada palabra. Que corresponde a darle un número a la mínima división de cada palabra.

Sin embargo los transformers y por tanto los LLMs, no representan así la información de las palabras, sino que lo hacen mediante `embeddings`.

Vamos a ver primero dos formas de representar las palabras dentro de los transformers, el `ordinal encoding` y el `one hot encoding`. Y viendo los problemas de estos dos tipos de representaciones podremos llegar hasta los `embeddings`.

## Ordinal encoding

Esta es la manera más básica de representar las palabras dentro de los transformers. Consiste en darle un número a cada palabra, o quedarnos con los números que ya tienen asignados los tokens.

Sin embargo este tipo de representación tiene dos problemas

 * Imaginemos que mesa corresponde al token 3, gato al token 1 y perro al token 2. Se podría llegar a suponer que `mesa = gato + perro`, pero no es así. No existe esa relación entre esas palabras. Incluso podríamos pensar que adjudicando los tokens correctos sí podría llegar a darse este tipo de relaciones. Sin embargo este pensamiento se viene abajo con las palabras que tienen más de un significado, como por ejemplo la palabra `banco`

 * El segundo problema es que las redes neuronales internamente hacen muchos cálculos numéricos, por lo que podría darse el caso en el que si mesa tiene el token 3, tenga internamente más importancia que la palabra gato que tiene el token 1.

De modo que este tipo de representación de las palabras se puede descartar muy rápidamente

## One hot encoding

Aquí lo que se hace es usar vectores de `N` dimensiones. Por ejemplo vimos que OpenAI tiene un vocabulario de `100277` tokens distintos. Por lo que si usamos `one hot encoding`, cada palabra se representaría con un vector de `100277` dimensiones.

Sin embargo el one hot encodding tiene otro dos grandes problemas

 * No tiene en cuenta la relación entre las palabras. Por lo que si tenemos dos palabras que son sinónimos, como por ejemplo `gato` y `felino`, tendríamos dos vectores distintos para representarlas. 
 En el lenguaje la relación entre las palabras es muy importante, y no tener en cuenta esta relación es un gran problema.

 * El segundo problema es que los vectores son muy grandes. Si tenemos un vocabulario de `100277` tokens, cada palabra se representaría con un vector de `100277` dimensiones. Esto hace que los vectores sean muy grandes y que los cálculos sean muy costosos. Además estos vectores van a ser todo ceros, excepto en la posición que corresponda al token de la palabra. Por lo que la mayoría de los cálculos van a ser multiplicaciones por cero, que son cálculos que no aportan nada. Así que vamos a tener un montón de memoria asignada a vectores en los que solo se tiene un 1 en una posición determinada.

## Word embeddings

En los word embeddings se intenta solucionar los problemas de los dos tipos de representaciones anteriores. Para ello se usan vectores de `N `dimensiones, pero en este caso no se usan vectores de 100277 dimensiones, sino que se usan vectores de muchas menos dimensiones. Por ejemplo veremos que OpenAI usa `1536` dimensiones.

Cada una de las dimensiones de estos vectores representan una característica de la palabra. Por ejemplo una de las dimensiones podría representar si la palabra es un verbo o un sustantivo. Otra dimensión podría representar si la palabra es un animal o no. Otra dimensión podría representar si la palabra es un nombre propio o no. Y así sucesivamente.

Sin embargo estas características no se definen a mano, sino que se aprenden de forma automática. Durante el entrenamiento de los transformers, se van ajustando los valores de cada una de las dimensiones de los vectores, de modo que se aprenden las características de cada una de las palabras.

Al hacer que cada una de las dimensiones de las palabras represente una característica de la palabra, se consigue que las palabras que tengan características similares, tengan vectores similares. Por ejemplo las palabras `gato` y `felino` tendrán vectores muy similares, ya que ambas son animales. Y las palabras `mesa` y `silla` tendrán vectores similares, ya que ambas son muebles.

En la siguiente imagen podemos ver una representación de 3 dimensiones depalabras, y podemos ver que todas las palabras relacionadas con `school` están cerca, todas las palabras relacionadas con `food` están cerca y todas las palabras relacionadas con `ball` están cerca.

![word_embedding_3_dimmension](http://maximofn.com/wp-content/uploads/2023/12/word_embedding_3_dimmension.webp)

Tener que cada una de las dimensiones de los vectores represente una característica de la palabra, consigue que podamos hacer operaciones con palabras. Por ejemplo si a la palabra `rey` se le resta la palabra `hombre` y se le suma la palabra `mujer`, obtenemos una palabra muy parecida a la palabra `reina`. Más adelante lo comprobaremos con un ejemplo

### Similitud entre palabras

Como cada una de las palabras se representa mediante un vector de N dimensiones, podemos calcular la similitud entre dos palabras. Para ello se usa la función de similitud del coseno o `cosine similarity`.

Si dos palabras están cercanas en el espacio vectorial, quiere decir que el álgulo que hay entre sus vectores es pequeño, por lo que su coseno es cercano a 1. Si hay un ángulo de 90 grados entre los vectores, el coseno es 0, es decir que no hay similitud entre las palabras. Y si hay un ángulo de 180 grados entre los vectores, el coseno es -1, es decir que las palabras son opuestas.

![cosine similarity](http://maximofn.com/wp-content/uploads/2023/12/cosine_similarity-scaled.webp)

### Ejemplo con embeddings de OpenAI

Ahora que sabemos lo que son los `embeddings`, veamos unos ejemplos con los `embeddings` que nos proporciona la `API` de `OpenAI`.

Para ello primero tenemos que tener instalado el paquete de `OpenAI`

```bash
pip install openai
```

Importamos las librerías necesarias

In [1]:
from openai import OpenAI
import torch
from torch.nn.functional import cosine_similarity

Usamos una `API key` de OpenAI. Para ello, nos dirigimos a la página de [OpenAI](https://openai.com/), y nos registramos. Una vez registrados, nos dirigimos a la sección de [API Keys](https://platform.openai.com/api-keys), y creamos una nueva `API Key`.

![open ai api key](https://raw.githubusercontent.com/maximofn/alfred/main/gifs/openaix2.gif)

In [2]:
api_key = "Pon aquí tu API key"

Seleccionamos que modelo de embeddings queremos usar. En este caso vamos a usar `text-embedding-ada-002` que es el que recomienda `OpenAI` en su documentación de [embeddings](https://platform.openai.com/docs/guides/embeddings/).

In [ ]:
model_openai = "text-embedding-ada-002"

Creamos un cliente de la `API`

In [ ]:
client_openai = OpenAI(api_key=api_key, organization=None)

Vamos a ver cómo son los `embeddings` de la palabra `Rey`

In [7]:
word = "Rey"
embedding_openai = torch.Tensor(client_openai.embeddings.create(input=word, model=model_openai).data[0].embedding)

embedding_openai.shape, embedding_openai

(torch.Size([1536]),
 tensor([-0.0103, -0.0005, -0.0189,  ..., -0.0009, -0.0226,  0.0045]))

Como vemos obtenemos un vector de `1536` dimensiones

### Operaciones con palabras

Vamos a obtener los embeddings de las palabras `rey`, `hombre`, `mujer` y `Reina`

In [19]:
embedding_openai_rey = torch.Tensor(client_openai.embeddings.create(input="rey", model=model_openai).data[0].embedding)
embedding_openai_hombre = torch.Tensor(client_openai.embeddings.create(input="hombre", model=model_openai).data[0].embedding)
embedding_openai_mujer = torch.Tensor(client_openai.embeddings.create(input="mujer", model=model_openai).data[0].embedding)
embedding_openai_reina = torch.Tensor(client_openai.embeddings.create(input="reina", model=model_openai).data[0].embedding)

In [20]:
embedding_openai_reina.shape, embedding_openai_reina

(torch.Size([1536]),
 tensor([-0.0110, -0.0084, -0.0115,  ...,  0.0082, -0.0096, -0.0024]))

Vamos a obtener el embedding resultante de restarle a `rey` el embedding de `hombre` y sumarle el embedding de `mujer`

In [21]:
embedding_openai = embedding_openai_rey - embedding_openai_hombre + embedding_openai_mujer

In [22]:
embedding_openai.shape, embedding_openai

(torch.Size([1536]),
 tensor([-0.0226, -0.0323,  0.0017,  ...,  0.0014, -0.0290, -0.0188]))

Por último comparamos el resultado obtenido con el embedding de `reina`. Para ello usamos la función de `cosine_similarity` que nos proporciona la librería `pytorch`

In [23]:
similarity_openai = cosine_similarity(embedding_openai.unsqueeze(0), embedding_openai_reina.unsqueeze(0)).item()

print(f"similarity_openai: {similarity_openai}")

similarity_openai: 0.7564167976379395


Como vemos es un valor muy cercano a 1, por lo que podemos decir que el resultado obtenido es muy parecido al embedding de `reina`

Si usamos palabras en inglés, obtenemos un resultado más cercano a 1

In [15]:
embedding_openai_rey = torch.Tensor(client_openai.embeddings.create(input="king", model=model_openai).data[0].embedding)
embedding_openai_hombre = torch.Tensor(client_openai.embeddings.create(input="man", model=model_openai).data[0].embedding)
embedding_openai_mujer = torch.Tensor(client_openai.embeddings.create(input="woman", model=model_openai).data[0].embedding)
embedding_openai_reina = torch.Tensor(client_openai.embeddings.create(input="queen", model=model_openai).data[0].embedding)

In [16]:
embedding_openai = embedding_openai_rey - embedding_openai_hombre + embedding_openai_mujer

In [17]:
similarity_openai = cosine_similarity(embedding_openai.unsqueeze(0), embedding_openai_reina.unsqueeze(0))
print(f"similarity_openai: {similarity_openai}")

similarity_openai: tensor([0.8849])


Esto es normal, ya que el modelo de OpenAi ha sido entrenado con más txtos en inglés que en español